In [1]:
import os
from timeit import default_timer as timer
import pandas as pd
from datetime import datetime, timedelta, date
from glob import glob
from pathlib import Path
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lag, lit
from pyspark.sql.window import Window
# from haversine import haversine_vector, Unit

In [2]:
# !!!! check that the timezone is correct for cuebiq data before running anything. See http://wrschneider.github.io/2019/09/01/timezones-parquet-redshift.html
assert spark.conf.get("spark.sql.session.timeZone") == 'Etc/UTC'

In [3]:
results_path = '/dbfs/mnt/Geospatial/results/'
start_date='2020-01-01'
end_date='2020-05-01'
source='cuebiq'
country='ID'
admin_id='ADM4_PCODE'
start_hour_day = 8
end_hour_day = 20

# Compute distance traveled

In [5]:
pings_geocoded = spark.read.parquet(os.path.join(results_path.replace("/dbfs/", ""),source, 'processed',country,'pings_geocoded/*.parquet'))

In [6]:
pings_geocoded = pings_geocoded.where((pings_geocoded.time >= start_date) & (pings_geocoded.time <= end_date))

In [7]:
# return distance in km. input should be in radians
def spark_haversine(lat_x, lon_x, lat_y, lon_y):
  d_lat = lat_y - lat_x 
  d_lon = lon_y - lon_x
  area = F.sin(d_lat/2)**2 + F.cos(lat_x) * F.cos(lat_y) * F.sin(d_lon/2)**2
  return 2 * F.asin(F.sqrt(area)) * lit(6371)

In [8]:
pings_geocoded = pings_geocoded.drop('classification_type','ADM4_PCODE')
w = Window().partitionBy("cuebiq_id").orderBy("time")
pings_geocoded = pings_geocoded.withColumn("lat_rad", F.radians(col("latitude"))).withColumn("lon_rad", F.radians(col("longitude")))
pings_geocoded = pings_geocoded.withColumn("distance", spark_haversine(F.col("lat_rad"), F.col("lon_rad"), lag("lat_rad", 1).over(w), lag("lon_rad", 1).over(w)))
pings_geocoded = pings_geocoded.dropna()
ping_geocoded = pings_geocoded.withColumn("hour", F.hour('time'))
ping_geocoded = ping_geocoded.drop("lat_rad", "lon_rad")

In [9]:
# https://stackoverflow.com/questions/46845672/median-quantiles-within-pyspark-groupby
magic_percentile = F.expr('percentile_approx(distance, 0.5, 100)')

In [10]:
tmp = ping_geocoded.withColumn("date", F.date_format('time','yyyy-MM-dd')).groupby("date", "cuebiq_id").agg( F.mean("distance").alias("mean_distance"),magic_percentile.alias("median_distance") , F.max("distance").alias("max_distance"), F.count("distance").alias("n_pings"))

In [11]:
distances_date_id = ping_geocoded.withColumn("date", F.date_format('time','yyyy-MM-dd')).groupby("date", "cuebiq_id").agg( F.mean("distance").alias("mean_distance"),magic_percentile.alias("median_distance") , F.max("distance").alias("max_distance"), F.count("distance").alias("n_pings"))

distances_daytime_id = ping_geocoded.filter((F.hour(col("time")) >= start_hour_day) & (F.hour(col("time")) < end_hour_day))\
                                            .withColumn("date", F.date_format('time','yyyy-MM-dd'))\
                                            .groupby("date", "cuebiq_id")\
                                            .agg(F.mean("distance").alias("mean_distance"),magic_percentile.alias("median_distance") , F.max("distance").alias("max_distance"), F.count("distance").alias("n_pings"))

distances_nighttime_id=ping_geocoded.filter(((F.hour(col("time")) >= end_hour_day) & (F.hour(col("time")) <= 23)) |((F.hour(col("time")) >= 0) & (F.hour(col("time")) < start_hour_day)))\
                                            .withColumn("date", F.date_format('time','yyyy-MM-dd')).groupby("date", "cuebiq_id")\
                                            .agg( F.mean("distance").alias("mean_distance"),magic_percentile.alias("median_distance") , F.max("distance").alias("max_distance"), F.count("distance").alias("n_pings"))


In [12]:

distances_date_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'distances_date_id.csv'), index=False)
distances_daytime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'distances_daytime_id.csv'), index = False)
distances_nighttime_id.toPandas().to_csv(os.path.join(results_path,source,'processed',country,'distances_nighttime_id.csv'), index = False)


# Figures